In [1]:
import pandas as pd
import requests
import math
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import nan

In [13]:
routes_list = list(df_routes['number'])


In [21]:
df_variants= get_variants()

In [22]:
df_variants

,key,name,Route,description
0,11-0-P,Portage-Kildonan to Polo Park,11.0,NaN
1,11-1-G,Portage-Kildonan to Glenway via,11.0,NaN
2,11-1-R,Portage-Kildonan to North Kildonan via Rothesay,11.0,NaN
3,11-0-*,Portage-Kildonan to Henderson,11.0,NaN
4,11-1-D,Portage-Kildonan to North Kildonan via Donwood,11.0,NaN
...,...,...,...,...
282,92-1-C,to Crossroads Station,92.0,NaN
283,92-1-K,to Kildonan Place,92.0,NaN
284,97-1-P,to Point Douglas,97.0,NaN
285,97-0-F,to Fife,97.0,NaN


In [24]:
df_variants.key.nunique()

273

# Code to Tally Bus Stops List Manually

In [ ]:
# Running the for loop for the balance bus variants

for bus_variant in bus_variants_list[100:]:
    response = requests.get(f"https://api.winnipegtransit.com/v3/stops.json?api-key=wPOIZFdePaV6BdttKD7C&variant={bus_variant}")
    
    # Creating a check on error server response
    # this condition will print out server error wherever any fetch finds error 
    
    if response.reason == 'OK':
        json_format = response.json()['stops']
    
        # Creating a temporary dataframe df_stops_each_variant which will updated with stops of bus variant on each loop
        # This will be used in concatenating each bus variant's stop info in the main dataframe df_stops
        df_stops_each_variant = pd.json_normalize(json_format)
         
        # Populating bus_variant column with bus variant number value to give the dataframe a unique identifier
        df_stops_each_variant['bus_variant'] = bus_variant
    
        # Concatenating each bus variant's info to main dataframe with bus stops information
        df_stops = pd.concat([df_stops, df_stops_each_variant], ignore_index=True)

    
    else:
        print(f"Bus variant {bus_variant} returned server error while fetching data")
        # Populating a list of bus variants for which destinations couldn't be fetched
        variants_without_stops.append(bus_variant)

In [ ]:
# Creating a function which will go inside in each bus_variants_chunks list elements
# And make an api call for each bus variant chunkwise

# Initializing with an empty pandas dataframe which will populate with each api call
df_stops_per_chunk = pd.DataFrame()


# This function takes in chunk_index(what chunk of the whole list to work on)
# Then for each chunk index makes api calls of bus variants in that chunk

def bus_stops_chunkwise(chunk_index):
    
    # Declaring the initialized dataframe variable df_stop_features as a global variable
    # Without it the function will consider it to be local variable and not work
    global df_stops_per_chunk
    
    df_stops_per_chunk = pd.DataFrame()
    
    # Running a for loop to make api call for each stop in the bus_stops_chunks list
    for variant in bus_variants_chunks[chunk_index]:
        
        try:
            
            response = requests.get(f"https://api.winnipegtransit.com/v3/stops.json?api-key=wPOIZFdePaV6BdttKD7C&variant={variant}")
            json_format = response.json()['stops']
            
            # If the search returns no errors or any empty value
            if json_format != []:
                
                # Recording the information got into a temporary frame df_each_stop_features
                df_each_variant_stops = pd.json_normalize(json_format)
                df_each_variant_stops['Bus_Variant'] = variant
            
           
            # If the api search returns a blank search, the variant will be recorded in the dataframe with empty values
            else:
                df_each_variant_stops.loc[len(df_each_variant_stops.index)] = [None, None,None,None,None,None,None,None,None,None,None,None,None,None, None,None, None, f'{variant}']
                print(f'Empty row added for variant {variant}')
   
        # If the api search returns any error, the variant will be recorded in the dataframe with empty values
        except:
            print(f'{variant} returned error in search. No values found for this variant')
            

         # Simulatenous concatanetion of each stop's info in df_stop_feature_chunk
        # This dataframe will contain records for each chunk of the big 4122 stops list
        df_stops_per_chunk = pd.concat([df_stops_per_chunk, df_each_variant_stops], ignore_index=True)


In [42]:
response = requests.get(f"https://api.winnipegtransit.com/v3/stops/10064/schedule.json?api-key=wPOIZFdePaV6BdttKD7C")
json_format = response.json()['stop-schedule']
df_each_variant_stops = pd.json_normalize(json_format['route-schedules'][0]['scheduled-stops'])

In [43]:
df_each_variant_stops

,key,cancelled,times.arrival.scheduled,times.arrival.estimated,times.departure.scheduled,times.departure.estimated,variant.key,variant.name,bus.key,bus.bike-rack,bus.wifi
0,22722390-21,false,2022-12-18T19:42:47,2022-12-18T19:43:57,2022-12-18T19:42:47,2022-12-18T19:43:57,16-0-B,Selkirk-Osborne to Tyndall Park via Burrows,860,false,false
1,22722391-23,false,2022-12-18T20:11:47,2022-12-18T20:11:47,2022-12-18T20:11:47,2022-12-18T20:11:47,16-0-M,Selkirk-Osborne to Tyndall Park via Manitoba,300,false,false
2,22722392-21,false,2022-12-18T20:40:47,2022-12-18T20:40:47,2022-12-18T20:40:47,2022-12-18T20:40:47,16-0-B,Selkirk-Osborne to Tyndall Park via Burrows,187,false,false
3,22722393-23,false,2022-12-18T21:09:47,2022-12-18T21:09:47,2022-12-18T21:09:47,2022-12-18T21:09:47,16-0-M,Selkirk-Osborne to Tyndall Park via Manitoba,303,false,false
4,22722394-21,false,2022-12-18T21:37:47,2022-12-18T21:37:47,2022-12-18T21:37:47,2022-12-18T21:37:47,16-0-B,Selkirk-Osborne to Tyndall Park via Burrows,435,true,false


In [44]:
json_format['route-schedules']

[{'route': {'key': 16,
   'number': 16,
   'name': 'Route 16 Selkirk-Osborne',
   'customer-type': 'regular',
   'coverage': 'regular',
   'badge-label': 16,
   'badge-style': {'class-names': {'class-name': ['badge-label', 'regular']},
    'background-color': '#ffffff',
    'border-color': '#d9d9d9',
    'color': '#000000'}},
  'scheduled-stops': [{'key': '22722390-21',
    'cancelled': 'false',
    'times': {'arrival': {'scheduled': '2022-12-18T19:42:47',
      'estimated': '2022-12-18T19:43:57'},
     'departure': {'scheduled': '2022-12-18T19:42:47',
      'estimated': '2022-12-18T19:43:57'}},
    'variant': {'key': '16-0-B',
     'name': 'Selkirk-Osborne to Tyndall Park via Burrows'},
    'bus': {'key': 860, 'bike-rack': 'false', 'wifi': 'false'}},
   {'key': '22722391-23',
    'cancelled': 'false',
    'times': {'arrival': {'scheduled': '2022-12-18T20:11:47',
      'estimated': '2022-12-18T20:11:47'},
     'departure': {'scheduled': '2022-12-18T20:11:47',
      'estimated': '2022-1

In [47]:
json_format['stop']['key']

10064

In [34]:
json_format['stop']

{'key': 10064,
 'name': 'Northbound Osborne at Glasgow',
 'number': 10064,
 'direction': 'Northbound',
 'side': 'Nearside',
 'street': {'key': 2715, 'name': 'Osborne Street', 'type': 'Street'},
 'cross-street': {'key': 1486, 'name': 'Glasgow Avenue', 'type': 'Avenue'},
 'centre': {'utm': {'zone': '14U', 'x': 633838, 'y': 5525742},
  'geographic': {'latitude': '49.86912', 'longitude': '-97.1375'}}}

In [2]:
df_stops = pd.read_csv('Bus_Stops.csv')

In [3]:
df_stops.number= df_stops.number.astype('Int64').astype('str')

In [4]:
bus_stops = list(set(df_stops.number))

In [5]:
for stop in bus_stops:
    response = requests.get(f"https://api.winnipegtransit.com/v3/stops/{stop}/schedule.json?api-key=wPOIZFdePaV6BdttKD7C")
    json_format = response.json()['stop-schedule']
    df_each_variant_stops = pd.json_normalize(json_format['route-schedules'][0]['scheduled-stops'])
    

IndexError: list index out of range

In [6]:
df_each_variant_stops

,key,cancelled,times.arrival.scheduled,times.arrival.estimated,times.departure.scheduled,times.departure.estimated,variant.key,variant.name,bus.key,bus.bike-rack,bus.wifi
0,22722914-7,false,2022-12-26T10:33:36,2022-12-26T10:33:36,2022-12-26T10:33:36,2022-12-26T10:33:36,10-1-WT,St. Boniface-Wolseley to Wolseley via Tache,171,false,false
1,22722915-7,false,2022-12-26T11:34:36,2022-12-26T11:34:36,2022-12-26T11:34:36,2022-12-26T11:34:36,10-1-WT,St. Boniface-Wolseley to Wolseley via Tache,171,false,false


In [69]:
df_each_variant_stops.cancelled.value_counts()

false    6
Name: cancelled, dtype: int64

In [7]:
stop

'20306'

In [8]:
response = requests.get(f"https://api.winnipegtransit.com/v3/stops/{stop}/schedule.json?api-key=wPOIZFdePaV6BdttKD7C")
json_format = response.json()['stop-schedule']
df_each_variant_stops = pd.json_normalize(json_format['route-schedules'][0]['scheduled-stops'])

IndexError: list index out of range

In [9]:
json_format

{'stop': {'key': 20306,
  'name': 'Eastbound Hamilton at Sonnichsen',
  'number': 20306,
  'direction': 'Eastbound',
  'side': 'Nearside Opposite',
  'street': {'key': 1617, 'name': 'Hamilton Avenue', 'type': 'Avenue'},
  'cross-street': {'key': 3383, 'name': 'Sonnichsen Place', 'type': 'Place'},
  'centre': {'utm': {'zone': '14U', 'x': 622970, 'y': 5528710},
   'geographic': {'latitude': '49.89813', 'longitude': '-97.28772'}}},
 'route-schedules': []}

In [13]:
json_format['route-schedules'] == {}

False